In [10]:
import scipy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import sys

In [11]:
train_file_path = 'lab2_train.csv'

train_data = pd.read_csv(train_file_path, delimiter=',')

test_file_path = 'lab2_test.csv'

test_data = pd.read_csv(test_file_path, delimiter=',')
print(train_data)
print(train_data.shape[0])
# Convert Timestamp column to datetime
#train_data['Timestamp'] = pd.to_datetime(train_data['Timestamp'], format='%d/%m/%Y %I:%M:%S %p')
#test_data['Timestamp'] =  pd.to_datetime(test_data['Timestamp'], format='%d/%m/%Y %I:%M:%S %p')

       user_from_id  user_to_id  is_like is_match
0              1136        3141    False    False
1              2424        3174    False    False
2              1300        3590    False    False
3               800        2736    False    False
4               883         437    False    False
...             ...         ...      ...      ...
76387          2376        3057    False    False
76388          1163         933    False    False
76389          2770        3324    False    False
76390           879         785    False    False
76391           291         470    False    False

[76392 rows x 4 columns]
76392


In [12]:
class HashFunction:
    """
    This HashFunction class can be used to create an unique hash given an alpha and beta.
    """
    def __init__(self, alpha, beta):
        self.alpha = alpha
        self.beta = beta

    def hashf(self, x, n):
        """
        Returns a hash given integers x and n.
        :param x: The value to be hashed
        :param n: The number of unique ids of all sets
        :return: The hashed value x given alpha and beta
        """
        
        hash_value = 0
        hash_value =  (self.alpha * x + self.beta) % n
        return hash_value

In [13]:
def compute_signature(hashes: list[HashFunction], ids: list[set[int]]):
    """
    This function will calculate the MinHash signature matrix from our sets of ids
    using the list of hash functions (hashes)
    :param hashes: The list of hash functions of arbitrary length
    :param ids: The list of sets of ids
    :return: The MinHash signature matrix for the given sets of ids
    """
    
    result = np.full((len(hashes), len(ids)), sys.maxsize)
    space = set().union(*ids)
    sorted_space = sorted(space)
    for p, ID in enumerate(sorted_space):
        arr = []
        for k, has in enumerate(hashes):
            arr.append(hashes[k].hashf(p, len(sorted_space)))
        for j in range(len(ids)):
            if (ID in ids[j]):
                for k, has in enumerate(hashes):
                    result[k,j] = min(arr[k], result[k,j])
    return result





    
    # START ANSWER
    # END ANSWER
    
    return result


preprocessing

In [14]:
#Remove the is_match column since that is not considered
train_data = train_data.drop('is_match', axis=1)

#sort the data by user_from_id
sorted_train_data = train_data.sort_values('user_from_id')

#filter the data by is_like = True, since we want to know for each user which users they have liked.
#The point point of a reccomender system is to recommend things and not to dissuade people from things.
filtered_sorted_train_data = sorted_train_data[sorted_train_data['is_like'] == True]

#group the data by user_from_id to create the sets of liked users for each user. 
#the aggregation created an outer list around the sets for some reason so I had to flatten it.
grouped_train_data = filtered_sorted_train_data.groupby('user_from_id')
set_grouped_train_data = grouped_train_data.agg({'user_to_id': lambda x: set(x)})
list_set_grouped_train_data = set_grouped_train_data.values.flatten().tolist()

#a print to show what the data looks like.
print(set_grouped_train_data)



               user_to_id
user_from_id             
1             {837, 3046}
2                  {3058}
3                  {1179}
4             {448, 3058}
5                    {15}
...                   ...
3682               {3185}
3688               {3238}
3689               {3366}
3707               {3444}
3714               {3568}

[2107 rows x 1 columns]


Prints the following things in order:
- the id of the first user
- the id of the second user
- the id's liked by the first user
- the id's liked by the second user
- the hash signature of the first user
- the hash signature of the second user

In [15]:
def print_stuff(res, hash_amount, similarity_ratio, neighbours_amount):
    for i in range(res.shape[1]):  # generate pairs
        neighbours = []
        for j in range(i + 1,res.shape[1]): 
            same = 0
            
            i_array = []
            j_array = []
            for k in range(hash_amount):
                    
                i_array.append(res[k][i])
                j_array.append(res[k][j])
                if (res[k][i] == res[k][j]):
                    same += 1
            if (same/hash_amount >= similarity_ratio and same/hash_amount < 1):
                neighbours.append((res[k][j], same/hash_amount, j))
                #print(i+1)
                #print(j+1)
                #print(set_grouped_train_data['user_to_id'].values[i])
                #print(set_grouped_train_data['user_to_id'].values[j])
                #print(i_array)
                #print(j_array)
                #print(" ")
            else:
                pass
        while (len(neighbours) > neighbours_amount):
                lowest_similarity_ratio = 2
                lowest_similarity_ratio_index = -1
                for k in range(len(neighbours)):
                    if (neighbours[k][1] < lowest_similarity_ratio):
                        lowest_similarity_ratio = neighbours[k][1]
                        lowest_similarity_ratio_index = k
                neighbours.pop(lowest_similarity_ratio_index)
        for k in range(len(neighbours)):
            neighbour_index = neighbours[k][2] 
            print(i+1)
            print(neighbour_index + 1)
            print(set_grouped_train_data['user_to_id'].values[i])
            print(set_grouped_train_data['user_to_id'].values[neighbour_index])
            print(i_array)
            print(neighbours[k][0])
            print(" ")
              

The performance of the recommender depends on the amount of hash functions used. the signature matrix returned by the compute_signature_matrix function is a matrix of size (n_hash_functions, m_users). so each column represents the hash of the set of "liked users" of a user. the chance that a specific minimum hash value is the same in different sets is equal to the jaccard similarity. This means that, assuming that we use enough hash functions, the amount of identical minimum values in two columns of the signature matrix is equal to the jaccard similarity of the two sets. The reason why we have to expirement with the amount of hash functions, is because using to few could cause two sets to appear more similar than they really are due to chance. This is demonstrated in the following examples as the fewer hash functions we use, the more non similar sets show up

In [16]:
hashes = []
#amount of hash functions
hash_amount = 10
similarity_ratio = 0.4
neighbours_amount = 3
for i in range(hash_amount):
    hashes.append(HashFunction(random.randint(1,10000), random.randint(1,10000)))

res = compute_signature(hashes, list_set_grouped_train_data)
print_stuff(res, hash_amount, similarity_ratio, neighbours_amount)

1
936
{837, 3046}
{3046}
[821, 379, 96, 1266, 136, 946, 1461, 414, 1017, 322]
322
 
1
946
{837, 3046}
{1249, 3046}
[821, 379, 96, 1266, 136, 946, 1461, 414, 1017, 322]
73
 
1
1458
{837, 3046}
{1820}
[821, 379, 96, 1266, 136, 946, 1461, 414, 1017, 322]
1460
 
1
1814
{837, 3046}
{1820}
[821, 379, 96, 1266, 136, 946, 1461, 414, 1017, 322]
1460
 
1
1893
{837, 3046}
{1820}
[821, 379, 96, 1266, 136, 946, 1461, 414, 1017, 322]
1460
 
2
4
{3058}
{448, 3058}
[830, 127, 1521, 162, 1384, 487, 862, 765, 1275, 725]
725
 
2
60
{3058}
{1840}
[830, 127, 1521, 162, 1384, 487, 862, 765, 1275, 725]
156
 
2
157
{3058}
{1648, 890, 1840}
[830, 127, 1521, 162, 1384, 487, 862, 765, 1275, 725]
156
 
2
520
{3058}
{1840}
[830, 127, 1521, 162, 1384, 487, 862, 765, 1275, 725]
156
 
2
1761
{3058}
{642, 351}
[830, 127, 1521, 162, 1384, 487, 862, 765, 1275, 725]
377
 
3
938
{1179}
{3595}
[176, 1270, 1215, 726, 598, 679, 766, 864, 735, 80]
1218
 
4
600
{448, 3058}
{3378, 1637}
[830, 91, 424, 162, 898, 487, 862, 735, 4

KeyboardInterrupt: 

In [ ]:
hashes = []
#amount of hash functions
hash_amount = 20
similarity_ratio = 0.4
neighbours_amount = 5
for i in range(hash_amount):
    hashes.append(HashFunction(random.randint(1,10000), random.randint(1,10000)))

res = compute_signature(hashes, list_set_grouped_train_data)
print_stuff(res, hash_amount, similarity_ratio, neighbours_amount)

1
936
{837, 3046}
{3046}
[341, 445, 81, 246, 384, 594, 1207, 680, 540, 363, 1210, 460, 1321, 291, 75, 163, 861, 1260, 889, 569]
[706, 445, 710, 1169, 384, 1439, 1694, 1287, 540, 363, 1502, 460, 1321, 291, 75, 163, 861, 1678, 1450, 569]
 
1
946
{837, 3046}
{1249, 3046}
[341, 445, 81, 246, 384, 594, 1207, 680, 540, 363, 1210, 460, 1321, 291, 75, 163, 861, 1260, 889, 569]
[706, 445, 710, 782, 384, 1253, 1205, 1014, 540, 363, 662, 460, 463, 291, 75, 163, 198, 382, 70, 560]
 
1
1937
{837, 3046}
{837}
[341, 445, 81, 246, 384, 594, 1207, 680, 540, 363, 1210, 460, 1321, 291, 75, 163, 861, 1260, 889, 569]
[341, 1522, 81, 246, 957, 594, 1207, 680, 832, 1466, 1210, 1646, 1608, 1378, 182, 691, 1419, 1260, 889, 1143]
 
1
1982
{837, 3046}
{837}
[341, 445, 81, 246, 384, 594, 1207, 680, 540, 363, 1210, 460, 1321, 291, 75, 163, 861, 1260, 889, 569]
[341, 1522, 81, 246, 957, 594, 1207, 680, 832, 1466, 1210, 1646, 1608, 1378, 182, 691, 1419, 1260, 889, 1143]
 
[341, 445, 81, 246, 384]
2
4
{3058}
{448, 30

In [ ]:
hashes = []
#amount of hash functions
hash_amount = 50
similarity_ratio = 0.4
neighbours_amount = 5
for i in range(hash_amount):
    hashes.append(HashFunction(random.randint(1,10000), random.randint(1,10000)))

res = compute_signature(hashes, list_set_grouped_train_data, neighbours_amount)
print_stuff(res, hash_amount, similarity_ratio, neighbours_amount)

TypeError: compute_signature() takes 2 positional arguments but 3 were given

In [ ]:
hashes = []
#amount of hash functions
hash_amount = 100
similarity_ratio = 0.4
neighbours_amount = 5
for i in range(hash_amount):
    hashes.append(HashFunction(random.randint(1,10000), random.randint(1,10000)))

res = compute_signature(hashes, list_set_grouped_train_data, neighbours_amount)
print_stuff(res, hash_amount, similarity_ratio, neighbours_amount)


1
936
{837, 3046}
{3046}
[981, 1375, 331, 279, 880, 6, 485, 119, 788, 1061, 1253, 139, 455, 30, 320, 1211, 62, 1095, 174, 384, 1015, 63, 1105, 63, 954, 29, 549, 1083, 975, 1498, 766, 709, 37, 384, 550, 732, 815, 608, 165, 885, 1298, 1182, 116, 585, 952, 462, 704, 684, 215, 758, 578, 757, 34, 825, 162, 1222, 783, 89, 1387, 1412, 381, 526, 441, 1415, 1483, 821, 155, 446, 689, 753, 539, 754, 630, 318, 1395, 71, 988, 919, 530, 714, 1185, 1129, 85, 333, 167, 588, 733, 287, 1068, 380, 457, 168, 893, 1003, 35, 517, 735, 128, 649, 1311]
[981, 1375, 729, 998, 880, 610, 485, 907, 1033, 1230, 1253, 139, 984, 196, 320, 1352, 622, 1095, 424, 794, 1147, 63, 1105, 1161, 954, 29, 549, 1252, 975, 1498, 766, 1005, 1489, 1230, 955, 1332, 1395, 608, 165, 1264, 1305, 1182, 116, 585, 952, 462, 1314, 684, 683, 912, 578, 886, 34, 1135, 348, 1222, 783, 647, 1553, 1700, 381, 1137, 841, 1415, 1646, 1408, 155, 753, 689, 753, 539, 1110, 1079, 856, 1398, 774, 1522, 1007, 1581, 1551, 1185, 1129, 648, 620, 958, 1125,